In [11]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim
from  tqdm import *

def conv_bn_relu(in_planes, out_planes, kernel=3, stride=1, padding=1):
    net = nn.Sequential(nn.Conv2d(in_planes, out_planes, kernel_size=kernel, stride=stride, padding=1),
                         nn.BatchNorm2d(num_features=out_planes),
                         nn.ReLU(True))
    return net;

class Net(nn.Module):
    
    def __init__(self, in_size=3):
        super(Net, self).__init__()
        
        self.net = nn.Sequential(conv_bn_relu(in_size, 32),
                      nn.MaxPool2d(2,2),
                      conv_bn_relu(32, 64),
                      nn.MaxPool2d(2, 2),
                      conv_bn_relu(64, 128),
                      conv_bn_relu(128, 64),
                      nn.UpsamplingBilinear2d(112), 
                      conv_bn_relu(64, 32),
                      nn.UpsamplingBilinear2d(224),
                      conv_bn_relu(32, 1),
                      nn.Sigmoid()
                      )

    def forward(self, input):
        return self.net(input)

net = Net()
# print(net)

In [12]:
import torch
import torch.nn as nn
import torch.utils.data as dt
from carvana_dataset import CarvanaDataset
from model import SegmenterModel
from torch.autograd import Variable
import torch.optim as optim
from tensorboardX import SummaryWriter
import os
from tqdm import *
import numpy as np

useCuda =True
n_epoch = 100
log = './log/'
train = './data/train/'
train_masks = './data/train_masks/'
test = './data/test/'
test_masks = './data/test_masks'

if os.path.exists(log) == False:
    os.mkdir(log)
tb_writer = SummaryWriter(log_dir='log')

if __name__ == '__main__':
    m = SegmenterModel()
    
    criterion = nn.BCELoss() 
    
    if criterion is None :
        print ('!!! Criterion is not implemented yet !!!')        
        
    optimizer = optim.Adam(m.parameters(), lr=0.001)

    if useCuda == True:
        m = m.cuda()
        criterion= criterion.cuda()

    ds = CarvanaDataset(train, train_masks)
    ds_test = CarvanaDataset(test, test_masks)

    dl      = dt.DataLoader(ds, shuffle=True, num_workers=4, batch_size=5)
    dl_test = dt.DataLoader(ds_test, shuffle=False, num_workers=4, batch_size=5)

    print (len(ds))
    global_iter = 0
    for epoch in range(0, n_epoch):
        print ("Current epoch: ", epoch)
        epoch_loss = 0
        m.train(True)
        for iter, (i, t) in enumerate(tqdm( dl) ):
            print (i.shape)
            i = Variable(i)
            t = Variable(t)
            if useCuda :
                i = i.cuda()
                t = t.cuda()
            o = m(i)
            loss = criterion(o, t)
            loss.backward()
            optimizer.step()

            global_iter += 1
            epoch_loss += loss.data[0]

        epoch_loss = epoch_loss / float(len(ds))
        print ("Epoch loss", epoch_loss)
        tb_writer.add_scalar('Loss/Train', epoch_loss, epoch)

        print ("Make test")
        test_loss = 0
        m.train(False)

        tb_out = np.random.choice(range(0, len(dl_test)), 3 )
        for iter, (i, t) in enumerate(tqdm(dl_test)):
            i = Variable(i, volatile = True)
            t = Variable(t, volatile = True)
            if useCuda :
                i = i.cuda()
                t = t.cuda()
            o = m(i)
            loss = criterion(o, t)
            test_loss += loss.data[0]

            for k, c in enumerate(tb_out):
                if c == iter:
                    tb_writer.add_image('Image/Test_input_%d'%k,  i[0].cpu(), epoch)  # Tensor
                    tb_writer.add_image('Image/Test_target_%d'%k, t[0].cpu(), epoch)  # Tensor
                    tb_writer.add_image('Image/Test_output_%d'%k, o[0].cpu(), epoch)  # Tensor

        test_loss = test_loss / float(len(ds_test))
        print ("Test loss", test_loss)
        tb_writer.add_scalar('Loss/Test', test_loss, epoch)


  0%|          | 0/916 [00:00<?, ?it/s]

4580
Current epoch:  0
torch.Size([5, 3, 224, 224])


RuntimeError: Need input.size[1] == 256 but got 3 instead.